In [61]:
#importing the usual library
import numpy as np
import pandas as pd
#scaling library
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
#train_test_split
from sklearn.model_selection import train_test_split
#DecisiontreeClassifier
from sklearn.tree import DecisionTreeRegressor
#Linear Regression
from sklearn.linear_model import LinearRegression
#KNeighbors
from sklearn.neighbors import KNeighborsRegressor

# Loading the Dataset

In [2]:
df=pd.read_csv('/kaggle/input/szeged-weather/weatherHistory.csv')
#loading the dataset
df

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


# Checking for the Preliminary Information

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Formatted Date            96453 non-null  object 
 1   Summary                   96453 non-null  object 
 2   Precip Type               95936 non-null  object 
 3   Temperature (C)           96453 non-null  float64
 4   Apparent Temperature (C)  96453 non-null  float64
 5   Humidity                  96453 non-null  float64
 6   Wind Speed (km/h)         96453 non-null  float64
 7   Wind Bearing (degrees)    96453 non-null  float64
 8   Visibility (km)           96453 non-null  float64
 9   Loud Cover                96453 non-null  float64
 10  Pressure (millibars)      96453 non-null  float64
 11  Daily Summary             96453 non-null  object 
dtypes: float64(8), object(4)
memory usage: 8.8+ MB


# Checking for the Misisng Value in the Dataset

In [4]:
df.isna().sum()

Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64

# Preprocessing the Dataset

In [62]:
def preprocess_inputs(df,scaler):
    df=df.copy()
    #dropping the summary and daily summary column from the dataset
    df=df.drop(['Summary','Daily Summary'],axis=1)
    #filling the mising value in Precip Type column
    
    df['Precip Type']=df['Precip Type'].fillna(df['Precip Type'].mode()[0])
    df['Formatted Year']=df['Formatted Date'].apply(lambda x:x[0:4]).astype(np.int)
    df['Formatted Month']=df['Formatted Date'].apply(lambda x:x[5:7]).astype(np.int)
    df['Formatted Day']=df['Formatted Date'].apply(lambda x:x[8:10]).astype(np.int)
    df['Formatted Hour']=df['Formatted Date'].apply(lambda x:x[10:12]).astype(np.int)
    df=df.drop('Formatted Date',axis=1)
    #encoding precip type column
    df['Precip Type']=df['Precip Type'].apply(lambda x:1 if x=='snow' else 0)
    
    
    
    df=df.drop('Loud Cover',axis=1)
    #splitting the data between target and feature
    y=df['Visibility (km)']
    x=df.drop('Visibility (km)',axis=1)

    x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)
    
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)
    
    
    return x_train,x_test,y_train,y_test

In [70]:
x_train,x_test,y_train,y_test=preprocess_inputs(df,MinMaxScaler())
print(x_test.shape)
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

(28936, 11)
(67517, 11)
(67517,)
(28936,)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depreca

In [64]:
models={
    'Linear Regression':LinearRegression(),
    'KN Neighbors':KNeighborsRegressor(),
    'Decision Tree Classifier':DecisionTreeRegressor()
}

In [71]:
for name,model in models.items():
    model.fit(x_train,y_train)
    print(name)
    print(model.score(x_test,y_test))

Linear Regression
0.22540349659716785
KN Neighbors
0.6487503440488336
Decision Tree Classifier
0.4272357349842104


* MinMaxScaler is the best scaler
* KN NeighborsRegressor is the best model

In [38]:
df['Formatted Date'].apply(lambda x:x[8:10]).astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


0        1
1        1
2        1
3        1
4        1
        ..
96448    9
96449    9
96450    9
96451    9
96452    9
Name: Formatted Date, Length: 96453, dtype: int64

In [48]:
x['Loud Cover'].unique()

array([0.])